<a href="https://colab.research.google.com/github/monu322/CE888/blob/main/Assg1/CE888_Assg1_TweetEval_John.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd

import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer


#importing sentiment data
tweets=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_text.txt', header=None)
print('Training tweets', tweets.shape)

tweets_sentiment=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_labels.txt', header=None)
print('Training labels', tweets_sentiment.shape)

tweets_test=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_text.txt', header=None)
print('Test tweets',tweets_test.shape)

tweets_test_sentiment=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_labels.txt', header=None)
print('Test labels',tweets_test_sentiment.shape)

X0=tweets[0]
X=X0
X_test=tweets_test[0]

y0=tweets_sentiment[0]
y=y0
y_test=tweets_test_sentiment[0]





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Training tweets (45615, 2)
Training labels (45615, 1)
Test tweets (12284, 1)
Test labels (12284, 1)


In [22]:
#Data pre processing, stemming and vectorization

stop_words=stopwords.words('english')
stemmer=PorterStemmer()

import re
cleaned_data=[]
cleaned_data_test=[]

for i in range(len(X)):
   tweet=re.sub('[^a-zA-Z]',' ',X.iloc[i])
   tweet=tweet.lower().split()
   tweet=[stemmer.stem(word) for word in tweet if (word not in stop_words)]
   tweet=' '.join(tweet)
   cleaned_data.append(tweet)


for i in range(len(X_test)):
   tweet=re.sub('[^a-zA-Z]',' ',X_test.iloc[i])
   tweet=tweet.lower().split()
   tweet=[stemmer.stem(word) for word in tweet if (word not in stop_words)]
   tweet=' '.join(tweet)
   cleaned_data_test.append(tweet)



from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000)
X_fin=cv.fit_transform(cleaned_data).toarray()
X_fin_test=cv.fit_transform(cleaned_data_test).toarray()


#Multinomial Naive Bayes Classification, Sentiment
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()

model.fit(X_fin,y)



y_pred=model.predict(X_fin_test)
from sklearn.metrics import classification_report
cf=classification_report(y_test,y_pred)
print(cf)



              precision    recall  f1-score   support

           0       0.35      0.24      0.29      3972
           1       0.49      0.64      0.55      5937
           2       0.18      0.14      0.15      2375

    accuracy                           0.41     12284
   macro avg       0.34      0.34      0.33     12284
weighted avg       0.38      0.41      0.39     12284



In [23]:
#Random Forest classifier, sentiment
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(class_weight='balanced', n_estimators=100)

model.fit(X_fin,y)

y_pred=model.predict(X_fin_test)
from sklearn.metrics import classification_report
cf=classification_report(y_test,y_pred)
print(cf)

              precision    recall  f1-score   support

           0       0.41      0.23      0.29      3972
           1       0.50      0.71      0.59      5937
           2       0.16      0.10      0.12      2375

    accuracy                           0.44     12284
   macro avg       0.35      0.35      0.34     12284
weighted avg       0.40      0.44      0.40     12284



In [24]:
!pip install transformers

In [25]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request



In [26]:
#roberta base, sentiment
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


X = X0[:1000]
y = y0[:1000]
y_pred = []

prediction_score = 0

for i in range(len(X)):
  print(i, X[i])
  text = X[i]
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  sentiment = ranking[0]
  y_pred.append(sentiment)

cf=classification_report(y,y_pred)
print(cf)



0 "QT @user In the original draft of the 7th book, Remus Lupin survived the Battle of Hogwarts. #HappyBirthdayRemusLupin"
1 "Ben Smith / Smith (concussion) remains out of the lineup Thursday, Curtis #NHL #SJ"
2 Sorry bout the stream last night I crashed out but will be on tonight for sure. Then back to Minecraft in pc tomorrow night.
3 Chase Headley's RBI double in the 8th inning off David Price snapped a Yankees streak of 33 consecutive scoreless innings against Blue Jays
4 @user Alciato: Bee will invest 150 million in January, another 200 in the Summer and plans to bring Messi by 2017"
5 @user LIT MY MUM 'Kerry the louboutins I wonder how many Willam owns!!! Look Kerry Warner Wednesday!'
6 "\"""" SOUL TRAIN\"""" OCT 27 HALLOWEEN SPECIAL ft T.dot FINEST rocking the mic...CRAZY CACTUS NIGHT CLUB ..ADV ticket $10 wt out costume $15..."
7 So disappointed in wwe summerslam! I want to see john cena wins his 16th title
8 "This is the last Sunday w/o football .....,NFL is back baby"
9 @user

In [27]:
#importing offesive dataset

tweets=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_text.txt', header=None)
print('Training tweets', tweets.shape)

tweets_sentiment=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_labels.txt', header=None)
print('Training labels', tweets_sentiment.shape)

tweets_test=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt', header=None)
print('Test tweets',tweets_test.shape)

tweets_test_sentiment=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt', header=None)
print('Test labels',tweets_test_sentiment.shape)

X0=tweets[0]
X=X0
X_test=tweets_test[0]

y0=tweets_sentiment[0]
y=y0
y_test=tweets_test_sentiment[0]



Training tweets (11916, 51)
Training labels (11916, 1)
Test tweets (860, 3)
Test labels (860, 1)


In [28]:
#Data pre processing, stemming and vectorization

stop_words=stopwords.words('english')
stemmer=PorterStemmer()

import re
cleaned_data=[]
cleaned_data_test=[]

for i in range(len(X)):
   tweet=re.sub('[^a-zA-Z]',' ',X.iloc[i])
   tweet=tweet.lower().split()
   tweet=[stemmer.stem(word) for word in tweet if (word not in stop_words)]
   tweet=' '.join(tweet)
   cleaned_data.append(tweet)


for i in range(len(X_test)):
   tweet=re.sub('[^a-zA-Z]',' ',X_test.iloc[i])
   tweet=tweet.lower().split()
   tweet=[stemmer.stem(word) for word in tweet if (word not in stop_words)]
   tweet=' '.join(tweet)
   cleaned_data_test.append(tweet)



from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000)
X_fin=cv.fit_transform(cleaned_data).toarray()
X_fin_test=cv.fit_transform(cleaned_data_test).toarray()


#Multinomial Naive Bayes Classification, offensive
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()

model.fit(X_fin,y)



y_pred=model.predict(X_fin_test)
from sklearn.metrics import classification_report
cf=classification_report(y_test,y_pred)
print(cf)



              precision    recall  f1-score   support

           0       0.73      0.41      0.52       620
           1       0.29      0.62      0.39       240

    accuracy                           0.47       860
   macro avg       0.51      0.51      0.46       860
weighted avg       0.61      0.47      0.49       860



In [29]:
#Random Forest classifier, offensive
model = RandomForestClassifier(class_weight='balanced', n_estimators=100)
model.fit(X_fin,y)

y_pred=model.predict(X_fin_test)
cf=classification_report(y_test,y_pred)
print(cf)

              precision    recall  f1-score   support

           0       0.69      0.56      0.62       620
           1       0.24      0.36      0.29       240

    accuracy                           0.50       860
   macro avg       0.47      0.46      0.45       860
weighted avg       0.57      0.50      0.53       860



In [30]:
task='offensive'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


X = X0[:1000]
y = y0[:1000]
y_pred = []

prediction_score = 0

for i in range(len(X)):
  print(i, X[i])
  text = X[i]
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  sentiment = ranking[0]
  y_pred.append(sentiment)

  

cf=classification_report(y,y_pred)
print(cf)




0 @user Bono... who cares. Soon people will understand that they gain nothing from following a phony celebrity. Become a Leader of your people instead or help and support your fellow countrymen.
1 @user Eight years the republicans denied obama’s picks. Breitbarters outrage is as phony as their fake president.
2 @user Get him some line help. He is gonna be just fine. As the game went on you could see him progressing more with his reads. He brought what has been missing. The deep ball presence. Now he just needs a little more time
3 @user @user She is great. Hi Fiona!
4 @user She has become a parody unto herself? She has certainly taken some heat for being such an....well idiot. Could be optic too  Who know with Liberals  They're all optics.  No substance
5 @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user This is the VetsResistSquadron"" is Bullshit.. They are girl scout veterans, I have never met any other veterans or served with anyone that was 

In [31]:
#importing emotion dataset

tweets=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_text.txt', header=None)
print('Training tweets', tweets.shape)

tweets_sentiment=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_labels.txt', header=None)
print('Training labels', tweets_sentiment.shape)

tweets_test=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt', header=None)
print('Test tweets',tweets_test.shape)

tweets_test_sentiment=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_labels.txt', header=None)
print('Test labels',tweets_test_sentiment.shape)

X0=tweets[0]
X=X0
X_test=tweets_test[0]

y0=tweets_sentiment[0]
y=y0
y_test=tweets_test_sentiment[0]



Training tweets (3257, 1)
Training labels (3257, 1)
Test tweets (1421, 1)
Test labels (1421, 1)


In [32]:
#Data pre processing, stemming and vectorization

stop_words=stopwords.words('english')
stemmer=PorterStemmer()

import re
cleaned_data=[]
cleaned_data_test=[]

for i in range(len(X)):
   tweet=re.sub('[^a-zA-Z]',' ',X.iloc[i])
   tweet=tweet.lower().split()
   tweet=[stemmer.stem(word) for word in tweet if (word not in stop_words)]
   tweet=' '.join(tweet)
   cleaned_data.append(tweet)


for i in range(len(X_test)):
   tweet=re.sub('[^a-zA-Z]',' ',X_test.iloc[i])
   tweet=tweet.lower().split()
   tweet=[stemmer.stem(word) for word in tweet if (word not in stop_words)]
   tweet=' '.join(tweet)
   cleaned_data_test.append(tweet)



from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000)
X_fin=cv.fit_transform(cleaned_data).toarray()
X_fin_test=cv.fit_transform(cleaned_data_test).toarray()


#Multinomial Naive Bayes Classification, emotion
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()

model.fit(X_fin,y)



y_pred=model.predict(X_fin_test)
from sklearn.metrics import classification_report
cf=classification_report(y_test,y_pred)
print(cf)



              precision    recall  f1-score   support

           0       0.38      0.34      0.36       558
           1       0.25      0.15      0.19       358
           2       0.08      0.30      0.13       123
           3       0.28      0.19      0.23       382

    accuracy                           0.25      1421
   macro avg       0.25      0.25      0.23      1421
weighted avg       0.29      0.25      0.26      1421



In [33]:
#Random Forest classifier, emotion
model = RandomForestClassifier(class_weight='balanced', n_estimators=100)
model.fit(X_fin,y)

y_pred=model.predict(X_fin_test)
cf=classification_report(y_test,y_pred)
print(cf)

              precision    recall  f1-score   support

           0       0.41      0.53      0.46       558
           1       0.25      0.17      0.20       358
           2       0.11      0.06      0.08       123
           3       0.30      0.31      0.31       382

    accuracy                           0.34      1421
   macro avg       0.27      0.27      0.26      1421
weighted avg       0.32      0.34      0.32      1421



In [34]:
task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


X = X0[:1000]
y = y0[:1000]
y_pred = []

prediction_score = 0

for i in range(len(X)):
  print(i, X[i])
  text = X[i]
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  sentiment = ranking[0]
  y_pred.append(sentiment)

  

cf=classification_report(y,y_pred)
print(cf)

0 “Worry is a down payment on a problem you may never have'.  Joyce Meyer.  #motivation #leadership #worry
1 My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs
2 No but that's so cute. Atsu was probably shy about photos before but cherry helped her out uwu
3 Rooneys fucking untouchable isn't he? Been fucking dreadful again, depay has looked decent(ish)tonight
4 it's pretty depressing when u hit pan on ur favourite highlighter
5 @user but your pussy was weak from what I heard so stfu up to me bitch . You got to threaten him that your pregnant .
6 Making that yearly transition from excited and hopeful college returner to sick and exhausted pessimist. #college
7 Tiller and breezy should do a collab album. Rapping and singing prolly be fire
8 @user broadband is shocking regretting signing up now #angry #shouldofgonewithvirgin
9 @user Look at those teef! #growl
10 @user @user USA was embarrassing to watch. When was the last time you guys won a